# C1 W4 Group 8 - Task 1


In [ ]:
from PIL import Image
from tqdm import tqdm
import cv2
import numpy as np
from src.data import AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST
from src.paths import BBDD_PATH, QSD1_W4_PATH, QSD1_NON_AUGMENTED_W4_PATH, WEEK_4_RESULTS_PATH
import matplotlib.pyplot as plt

In [ ]:
BBDD_PATH_LIST = sorted(BBDD_PATH.glob("*.jpg"))
QSD1_W4_PATH_LIST = sorted(QSD1_W4_PATH.glob("*.jpg"))
QSD1_NON_AUGMENTED_W4_PATH_LIST = sorted(QSD1_NON_AUGMENTED_W4_PATH.glob("*.jpg"))

In [ ]:
database_image_PIL_list = [Image.open(db_img_path) for db_img_path in BBDD_PATH_LIST]  # Load once
for idx, db_img in enumerate(database_image_PIL_list):
    assert db_img.filename.endswith(f"{idx}.jpg")

In [ ]:
query_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(query_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [ ]:
non_augmented_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_NON_AUGMENTED_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(non_augmented_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [ ]:
for file, augmentation, frames, gt in zip(QSD1_W4_PATH_LIST, AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST):
    frame_info = "; ".join([f"({frame[0]:.2f}, {frame[1]})" for frame in frames])
    print(f"File={file.stem}, Aug={augmentation}, Frames={frame_info}, GT={gt}")

## Denoising over the images

In [ ]:
from src.denoising import denoise_image

In [ ]:
# Median filter
denoised_query_d1_PIL_list = [denoise_image(image) for image in query_d1_image_PIL_list]

In [ ]:
def plot_images_with_multiple_filters(original_images, noisy_images, denoised_images, save_path):
    num_images = len(original_images)
    num_cols = 3
    fig, axes = plt.subplots(num_images, num_cols, figsize=(25, 5 * num_images))  # +1 for original image

    for i in range(num_images):
        original_image = original_images[i]
        noisy_image = noisy_images[i]

        axes[i, 0].imshow(original_image)
        axes[i, 0].set_title(f'Original Image {i+1}')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(noisy_image)
        axes[i, 1].set_title(f'Noisy image Image {i+1}')
        axes[i, 1].axis('off')

        axes[i, 2].imshow(denoised_images[i])
        axes[i, 2].set_title(f'Denoised {i+1}')
        axes[i, 2].axis('off')

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

In [ ]:
#plot_images_with_multiple_filters(non_augmented_d1_image_PIL_list, query_d1_image_PIL_list,
                                   denoised_query_d1_PIL_list, WEEK_4_RESULTS_PATH / "Task_1"/ 'denoised_images_plot.png')

## Background removal

In [ ]:
from src.old_background import get_painting_masks, crop_image_by_mask

In [ ]:
def crop_paintings_from_image(image, ground_truth):
    image_np = np.array(image)
    cropped_images = []

    for points in (entry[1] for entry in ground_truth):
        src_points = np.array(points, dtype=np.float32)
        width, height = map(int, [np.linalg.norm(src_points[1] - src_points[0]), np.linalg.norm(src_points[2] - src_points[1])])
        dst_points = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

        M = cv2.getPerspectiveTransform(src_points, dst_points)
        cropped_images.append(Image.fromarray(cv2.warpPerspective(image_np, M, (width, height))))

    return cropped_images

In [ ]:
cropped_query_image_list_d1 = []
pbar = tqdm(zip(sorted(QSD1_W4_PATH.glob("*.jpg")), denoised_query_d1_PIL_list, FRAMES_QSD1_W4_LIST))
for name, image, predicted_mask in pbar:
    pbar.set_description(f"Splitting {name.stem}")
    cropped_query_image_list_d1.append(crop_paintings_from_image(image, predicted_mask))

In [ ]:
n_rows = len(cropped_query_image_list_d1)

fig, axes = plt.subplots(n_rows, 2, figsize=(6, n_rows *1.5))

for ax, l in zip(axes, cropped_query_image_list_d1):
    ax[0].imshow(l[0]); ax[0].axis('off')
    if len(l) > 1:
        ax[1].imshow(l[1])
    ax[1].axis('off')

plt.tight_layout()
plt.show()

## Keypoint extraction

In [ ]:
from src.descriptors import SIFTDescriptor, ORBDescriptor, HOGDescriptor, ImageRetrievalSystem

In [ ]:
# denoise also dataset ?
denoise_database_image = [np.array(denoise_image(image)) for image in database_image_PIL_list]

In [ ]:
descriptors = {
    # 'SIFT': SIFTDescriptor(max_features=500),

    'HOG13': HOGDescriptor(pixels_per_cell=12, cells_per_block=4),


    # 'ORB': ORBDescriptor(),
}
descriptor_path = WEEK_4_RESULTS_PATH / 'Task_1' / 'descriptors'  

retrieval_system = ImageRetrievalSystem(descriptors, descriptor_path)

In [ ]:
# Set number of top similar images to retrieve
K = 10

results = {}
g_0 = {}

for descriptor_name in descriptors.keys():
    print(f"\nRetrieving similar images using {descriptor_name} descriptor:")
    results[descriptor_name] = retrieval_system.retrieve_similar_images(descriptor_name, denoise_database_image, cropped_query_image_list_d1, GT_QSD1_W4_LIST, K)

In [ ]:
from src.metrics import MeanAveragePrecisionAtK

In [ ]:
map = MeanAveragePrecisionAtK()

In [ ]:
GT_mod = []
for element in GT_QSD1_W4_LIST:
    if len(element) == 2:
        GT_mod.extend([[element[0]], [element[1]]])
    else:
        GT_mod.append([element[0]])

results_mod = {}
for descriptor_name in descriptors.keys():
    results_mod[descriptor_name] = []
    for element in results[descriptor_name]:
        if len(element) == 2:
            results_mod[descriptor_name].extend([[element[0][0][0]], [element[1][0][0]]])
        else:
            results_mod[descriptor_name].append([element[0][0][0]])

print(GT_mod,"\n"+"#" * 40)
print(f'\n{"#"*40}\n'.join([str(result) for result in results_mod.values()]))

In [ ]:
for descriptor_name, results in results_mod.items():
    print(f"Results for {descriptor_name}:")
    results = map.compute(GT_mod, results, k=K)
    print(f"{results}\n{"#" * 40}\n")

In [ ]:
idx = 0
non_found_dist = []
for res, gt, gap in zip(results_mod, GT_mod, g_0):
    print(f"{idx} | Res: {res}, GT: {gt}, gaps: {gap[1]}")
    idx += 1

non_found_dist = []
for idx, (res, gt, gap) in enumerate(zip(results_mod, GT_mod, g_0)):
    gap_str = ', '.join([f"{g:.2f}" for g in gap[1]])  # Format gaps to 2 decimal places
    print(f"{idx:2} | Res: {res:<50} | GT: {gt:<10} | Gaps: [{gap_str}]")


In [ ]:
db_i = descriptors['SIFT'].compute(np.array(database_image_PIL_list[32]))
query_i = descriptors['SIFT'].compute(np.array(cropped_query_image_list_d1[4][0]))
print(db_i[0][0])

In [ ]:
from scipy.spatial.distance import cdist

distances = cdist(db_i[1], query_i[1], metric='euclidean')
min_distances = np.min(distances, axis=1)
mean = np.mean(min_distances)

print(mean)

db_i = descriptors['SIFT'].compute(np.array(database_image_PIL_list[177]))
distances = cdist(db_i[1], query_i[1], metric='euclidean')
min_distances = np.min(distances, axis=1)
mean = np.mean(min_distances)

print(mean)
